텍스트 파일과 PDF 파일 기반의 RAG 시스템을 JSON 데이터 형식으로 확장

정형화된 데이터(JSON)를 어떻게 RAG 시스템에 효율적으로 주입하고 검색하는지를 보여주며, 실무에서 DB나 API로부터 데이터를 가져올 때 가장 흔하게 사용하는 방식

In [ ]:
!pip install chromadb sentence-transformers pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.8 MB/s et

In [ ]:
import os, re, json, uuid    # uuid : 고유 식별 id 생성용
from typing import List   # type hint 기능 제공 (가독성)
from sentence_transformers import SentenceTransformer   # 문장 단위의 의미 임베딩 라이브러리
from chromadb import PersistentClient   #  CHROMA_DIR에 정의된 폴더에 벡터 데이터를 저장하여 프로그램이 종료되어도 유지

JSON_PATH = "sample.json"
CHROMA_DIR = ".chroma_json_demo"
COLLECTION = "json_docs"      # 지식 분리: JSON 데이터와 다른 형식(PDF, TXT)의 데이터를 명확하게 분리하여 검색의 정확도를 유지
MODEL_NAME = "all-MiniLM-L6-v2"

model = SentenceTransformer(MODEL_NAME) # 특징 추출기: 텍스트를 고차원 벡터로 변환하는 AI 모델
client = PersistentClient(path = CHROMA_DIR)
collection = client.get_or_create_collection(COLLECTION)

In [ ]:
def upsert_jsonFunc(json_path:str):
  if not os.path.exists(json_path):
    raise FileNotFoundError("파일없음")
  with open(json_path, 'r', encoding='utf-8', errors='ignore') as f:
    data = json.load(f) # ⭐ 핵심: json.load()를 사용해 JSON 파일 내용을 파이썬 (리스트 of 딕셔너리)로 변환
    if not data:
      print("자료 없음")
      return
  # 1. ID 생성(List Comprehension 사용)
  ids = [item.get("id", str(uuid.uuid4())) for item in data]  # item.get("id", str(uuid.uuid4())) : JSON 항목에 'id'가 있으면 그 값을 쓰고, 없으면 uuid로 새로운 고유 ID를 생성.
  # 2. Documents (임베딩할 텍스트) 생성: 제목과 내용을 합쳐서 하나의 텍스트로 만듦(List Comprehension 사용)
  docs = [f"{item.get('title', '')}, {item.get('content', '')}" for item in data]   # f"{item.get('title', '')}, {item.get('content', '')}" : title과 content 필드를 쉼표로 연결하여 하나의 긴 텍스트 청크를 만듦
  # 3. Metadatas (꼬리표) 생성: 제목과 출처(파일명) 저장
  metas = [{"title":item.get('title',''), "source":os.path.basename(json_path)}for item in data]  # "title"과 "source" 정보를 저장하여 나중에 검색 결과가 어디서 왔는지 근거를 제공
  # 4. 임베딩 벡터 생성
  embs = model.encode(docs, normalize_embeddings=True).tolist() # docs 리스트의 모든 텍스트를 한 번에 벡터로 변환

  # 5. DB에 저장
  collection.add(ids=ids, documents=docs, embeddings=embs, metadatas=metas) # IDs, 원본 텍스트(Documents), 벡터(Embeddings), 부가 정보(Metadatas)를 DB에 일괄 저장
  print(f'저장 완료')



def searchFunc(query:str, k:int):
  # 1. 쿼리 임베딩: 질문을 벡터로 변환 (모델, 정규화 모두 upsert 시와 동일)
  q_emb = model.encode([query], normalize_embeddings=True).tolist()
  # 2. ChromaDB 쿼리: 가장 유사한 k개의 결과를 검색
  res = collection.query(query_embeddings=q_emb, n_results=k)
  # 3. 결과 추출 및 출력 (docs, metas, dists)
  docs = res.get('documents', [[]])[0] # 예외 방지용 패턴
  metas = res.get('metadatas', [[]])[0]
  ids = res.get('ids', [[]])[0]
  dists = res.get('distances', [[]])[0]

  for i, (doc, meta, _id, dist) in enumerate(zip(docs, metas, ids, dists)):
      print(f'\n[{i}] id={_id}')
      print(f'source={meta.get("source")}, len={meta.get("len")}, distance={dist:.4f}')
      print(doc[:100] + ("..." if len(doc) > 100 else ""))

꼭 기억해야 할 핵심 내용
json.load(f)	 : JSON 데이터 파싱의 시작. 함수 덕분에 복잡한 JSON 형식이 파이썬에서 다루기 쉬운 dict나 list 형태로 변환.

별도의 Chunking 함수가 없는 이유	: 텍스트나 PDF와 달리, JSON 파일은 이미 각 항목(item)이 하나의 완성된 의미 단위 (하나의 청크)로 구조화됨. 따라서 별도의 split_paragraphFunc 대신, JSON의 각 item을 바로 청크로 사용.


docs 생성 (정보 병합)	: f"{item.get('title', '')}, {item.get('content', '')}" 이 부분이 매우 중요. 제목(title)과 본문(content) 필드를 하나의 텍스트로 합쳐서 임베딩. LLM에게 더 풍부한 문맥 정보를 제공하여 검색 정확도를 높임.


ID 처리	: JSON 데이터에 이미 id 필드가 있다면 그것을 사용하고, 없다면 uuid로 새로 생성하는 유연한 방식을 사용.

model.encode 재사용 : 임베딩 모델을 다시 정의하거나 로드하지 않고, 클래스 초기화 시 로드된 model 객체를 재사용함으로써 효율성

In [ ]:
if __name__ == "__main__":
    upsert_jsonFunc(JSON_PATH)
    print("\n검색 예 : ")
    searchFunc("노드와 포인터로 이루어진 자료구조 만세", k=3)

저장 완료

검색 예 : 

[0] id=p003
source=sample.json, len=None, distance=0.4880
정렬 알고리즘, 선택 정렬과 삽입 정렬은 단순하지만 느리고, 퀵 정렬은 평균적으로 빠른 성능을 보입니다.

[1] id=p002
source=sample.json, len=None, distance=0.6066
링크드 리스트, 링크드 리스트는 노드와 포인터로 이루어진 자료구조입니다. 삽입과 삭제가 효율적입니다.

[2] id=p001
source=sample.json, len=None, distance=0.6188
배열의 기본 개념, 배열은 같은 자료형의 데이터를 연속된 공간에 저장하는 자료구조입니다. 인덱스를 이용해 빠르게 접근할 수 있습니다.


핵심 내용
JSON 처리: json.load()를 사용하여 정형 데이터를 RAG에 통합하는 표준 방식.

Chunking 생략 이유: JSON은 이미 구조화된 데이터이므로, 각 항목(item)이 곧 하나의 의미 있는 청크(Chunk).

데이터 병합 (Docs): 제목과 내용을 합쳐서 임베딩함으로써, 텍스트의 문맥과 핵심 주제를 벡터에 더 잘 담아 검색의 정확도를 높힘.

컬렉션 분리: json_docs 컬렉션을 사용하여 다른 파일 형식의 데이터와 지식을 분리하는 관리.